### Итоговый проект курса "Машинное обучение в бизнесе"

### -- Автор: Шенк Евгений Станиславович

In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
import lightgbm as lgbm

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
#imputer
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv('./train.csv')
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
structures = pd.read_csv('./structures.csv')

data.head(3)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4659076 entries, 0 to 4659075
Data columns (total 6 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id                        int64  
 1   molecule_name             object 
 2   atom_index_0              int64  
 3   atom_index_1              int64  
 4   type                      object 
 5   scalar_coupling_constant  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 213.3+ MB


In [4]:
structures.head(3)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277


In [5]:
target = data['scalar_coupling_constant']

In [6]:
print(f'{data.shape[0]} rows in train data.')

print(f"{data['molecule_name'].nunique()} distinct molecules in train data.")
print(f"{structures['atom'].nunique()} unique atoms.")
print(f"{data['type'].nunique()} unique types.")

4659076 rows in train data.
85012 distinct molecules in train data.
5 unique atoms.
8 unique types.


In [7]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=['scalar_coupling_constant'], axis=1), target, test_size=0.25, random_state=2177)

### Feature engineering

In [8]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

In [9]:
data = map_atom_info(data, 0)
data = map_atom_info(data, 1)

In [10]:
data_p_0 = data[['x_0', 'y_0', 'z_0']].values
data_p_1 = data[['x_1', 'y_1', 'z_1']].values

data['dist'] = np.linalg.norm(data_p_0 - data_p_1, axis=1)

data['dist_x'] = (data['x_0'] - data['x_1']) ** 2
data['dist_y'] = (data['y_0'] - data['y_1']) ** 2
data['dist_z'] = (data['z_0'] - data['z_1']) ** 2

In [11]:
data['type_0'] = data['type'].apply(lambda x: x[0])
data['type_1'] = data['type'].apply(lambda x: x[1:])

### Encoder

In [12]:
# Data encoding
labelencoder = LabelEncoder()
features = ['atom_0', 'atom_1', 'type_0', 'type_1', 'type']
for f in features:
    labelencoder.fit(list(data[f].values))
    data[f] = labelencoder.transform(list(data[f].values))
    np.save(f'classes_{f}.npy', labelencoder.classes_)

### Model - LGBMRegressor

In [13]:
X1_train, X1_test, y1_train, y1_test = train_test_split(data.drop(columns=['scalar_coupling_constant'], axis=1), target, test_size=0.25, random_state=2177)

In [14]:
imp_features = ['atom_index_0', 'atom_index_1', 'type', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'dist', 'dist_x', 'dist_y', 'dist_z', 'type_0', 'type_1']

In [15]:
model = lgbm.LGBMRegressor(learning_rate=0.1,
                    n_estimators=500,
                    num_leaves=128,
                    reg_alpha=0.1,
                    reg_lambda=0.1)

In [16]:
%%time
model.fit(X1_train[imp_features], y1_train) 

Wall time: 41.2 s


LGBMRegressor(n_estimators=500, num_leaves=128, reg_alpha=0.1, reg_lambda=0.1)

In [17]:
pred = model.predict(X1_test[imp_features])

In [18]:
scores = cross_val_score(model, X1_train[imp_features], y1_train, cv=3, scoring='r2')
scores.mean()

0.9907120792849257

In [19]:
r2_score(y1_test, pred)

0.990676619045612

### Pipeline

In [20]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

class L_Encoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = X.name
        return self

    def transform(self, X):
        labelencoder = LabelEncoder()
        labelencoder.classes_ = np.load('classes.npy', allow_pickle=True)
        
        X = labelencoder.fit_transform(X)
        return pd.DataFrame(data=X, columns=[self.columns])
    
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
  
    def fit( self, X, y = None ):
        return self 

    def transform( self, X, y = None ):
        return X[self._feature_names] 
    
class Feat_eng_pipe(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = []
    
    def fit(self, X, y=None):
        return self
    
    def map_atom_info(self, df, structures, atom_idx):
        df = pd.merge(df, structures, how = 'left',
                      left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                      right_on = ['molecule_name',  'atom_index'])

        df = df.drop('atom_index', axis=1)
        df = df.rename(columns={'atom': f'atom_{atom_idx}',
                                'x': f'x_{atom_idx}',
                                'y': f'y_{atom_idx}',
                                'z': f'z_{atom_idx}'})
        return df
    
    def Load_structures(self):
        path = ['./models/structures.csv', './structures.csv']
        structures_load = None
        for p in path:
            try:
                structures_load = pd.read_csv(p)
                break
            except:
                continue
        return structures_load
    
    def Load_encoder_classes(self, feature):
        path = [f'./models/classes_{feature}.npy', f'./classes_{feature}.npy']
        cls = None
        for p in path:
            try:
                cls = np.load(p, allow_pickle=True)
                break
            except:
                continue
        return cls
    
    def transform( self, X, y = None ):
        import numpy as np
        import pandas as pd
        
        structures = self.Load_structures()
        X = self.map_atom_info(X, structures, 0)
        X = self.map_atom_info(X, structures, 1)
        
        X_p_0 = X[['x_0', 'y_0', 'z_0']].values
        X_p_1 = X[['x_1', 'y_1', 'z_1']].values

        X['dist'] = np.linalg.norm(X_p_0 - X_p_1, axis=1)
        X['dist_x'] = (X['x_0'] - X['x_1']) ** 2
        X['dist_y'] = (X['y_0'] - X['y_1']) ** 2
        X['dist_z'] = (X['z_0'] - X['z_1']) ** 2
        
        X['type_0'] = X['type'].apply(lambda x: x[0])
        X['type_1'] = X['type'].apply(lambda x: x[1:])
        
        from sklearn.preprocessing import LabelEncoder
        
        features = ['atom_0', 'atom_1', 'type_0', 'type_1', 'type']
        for f in features:
            labelencoder = LabelEncoder()
            labelencoder.classes_ = self.Load_encoder_classes(feature=f)
            X[f] = labelencoder.transform(list(X[f].values))
        
        feat = ['type', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'dist', 'dist_x', 'dist_y', 'dist_z', 'type_0', 'type_1']
        return X[feat]

In [21]:
imp_features = ['atom_index_0', 'atom_index_1', 'type', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'dist', 'dist_x', 'dist_y', 'dist_z', 'type_0', 'type_1']

In [22]:
no_enc_features = ['atom_index_0', 'atom_index_1']

In [23]:
f_selector = Pipeline([
                ('f_selector', FeatureSelector(no_enc_features))])

f_eng_pipe = Pipeline([
                ('Feat_eng_pipe', Feat_eng_pipe())
                ])

In [24]:
feats = FeatureUnion([('f_selector', f_selector),
                    ('f_eng_pipe', f_eng_pipe)])

feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

array([[16.        , 17.        ,  3.        , ...,  2.80680317,
         1.        ,  1.        ],
       [ 9.        ,  7.        ,  5.        , ...,  0.39189991,
         2.        ,  0.        ],
       [18.        ,  8.        ,  2.        , ...,  0.04919705,
         1.        ,  0.        ],
       ...,
       [12.        ,  0.        ,  2.        , ...,  0.24354726,
         1.        ,  0.        ],
       [14.        ,  3.        ,  2.        , ...,  3.41186882,
         1.        ,  0.        ],
       [ 9.        , 11.        ,  3.        , ...,  1.1969371 ,
         1.        ,  1.        ]])

In [25]:
%%time
pipeline = Pipeline([
    ('features', feats),
    ('regressor', lgbm.LGBMRegressor(learning_rate=0.01,
                                    n_estimators=1500,
                                    num_leaves=128,
                                    reg_alpha=0.1,
                                    reg_lambda=0.1)),
])

pipeline.fit(X_train, y_train)

pred = pipeline.predict(X_test)
r2_score(y_test, pred)

Wall time: 3min 58s


0.9896726280263306

In [26]:
with open("Course_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)